# Bended modules 

At the very base of the `torchbend` modules lies the `BendedModule`, a wrapper for `torch.nn.Module` that uses the `torchbend` improved tracer to provide a handy interface for weight bending, interpolation, and activation retrieval / and bending. 

## BendedModule and evolved tracing.

Let's see how to bend this simple module : 

In [ ]:
import torch, torch.nn as nn
import torchbend as tb

class Stanley(nn.Module):
    def __init__(self):
        self.conv_modules = nn.Sequential(
            nn.Conv1d(1, 4), 
            nn.Conv1d(4, 8)
        )
        self.batch_norm = nn.BatchNorm1d(8)
        self.nnlin = nn.Sigmoid()
    
    def forward(self, x):
        out = self.conv_modules(x)
        out = self.batch_norm(out)
        out = self.nnlin(out)
        return out

    def forward_nobatch(self, x):
        out = self.conv_modules(x)
        out = self.nnlin(out)
        return out

module = Stanley()